# Data Wrangling
***
Trip data for Bay Wheels is publicly available per month. We download it from the repo and clean the data where needed.

In [1]:
import numpy as np
import pandas as pd

import requests
import zipfile
import os
import io

## 1. Gather Data
***
The data can be found [here](https://s3.amazonaws.com/baywheels-data/index.html).

In [2]:
# make a data-folder
data_folder = 'data'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

In [3]:
# target url to download from
url = 'https://s3.amazonaws.com/baywheels-data/202001-baywheels-tripdata.csv.zip'

# define output_locations
output_file = url.split('/')[-1][:-4] 

# download
r = requests.get(url, stream=True)
with zipfile.ZipFile(io.BytesIO(r.content)) as myzip:
    myzip.extract(output_file, data_folder)

In [4]:
# load
df = pd.read_csv(data_folder + '/' + output_file)
print(df.shape)
df.head()

(295854, 14)


/Users/gijsvanpamelen/projects/bay_wheels_trips/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,rental_access_method
0,83118,2020-01-31 15:23:47.7330,2020-02-01 14:29:06.2630,400.0,Buchanan St at North Point St,37.804272,-122.433537,400.0,Buchanan St at North Point St,37.804272,-122.433537,13052,Customer,NaN
1,68239,2020-01-31 15:40:31.6160,2020-02-01 10:37:51.0000,99.0,Folsom St at 15th St,37.767037,-122.415443,99.0,Folsom St at 15th St,37.767037,-122.415443,12235,Subscriber,NaN
2,55407,2020-01-31 17:48:16.6350,2020-02-01 09:11:44.3170,197.0,El Embarcadero at Grand Ave,37.808848,-122.249680,197.0,El Embarcadero at Grand Ave,37.808848,-122.249680,12822,Customer,NaN
3,54929,2020-01-31 17:53:03.4130,2020-02-01 09:08:32.6340,197.0,El Embarcadero at Grand Ave,37.808848,-122.249680,197.0,El Embarcadero at Grand Ave,37.808848,-122.249680,11705,Customer,NaN
4,55700,2020-01-31 17:12:33.4600,2020-02-01 08:40:53.6460,12.0,Pier 1/2 at The Embarcadero,37.796389,-122.394586,371.0,Lombard St at Columbus Ave,37.802746,-122.413579,3673,Customer,NaN


## 2. Assess Data
***
Data assets ([source](https://www.lyft.com/bikes/bay-wheels/system-data)):
* Trip Duration (seconds)
* Start Time and Date
* End Time and Date
* Start Station ID
* Start Station Name
* Start Station Latitude
* Start Station Longitude
* End Station ID
* End Station Name
* End Station Latitude
* End Station Longitude
* Bike ID
* User Type (Subscriber or Customer – “Subscriber” = Member or “Customer” = Casual)

### 2.1 High level exploration

In [5]:
df.shape

(295854, 14)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295854 entries, 0 to 295853
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             295854 non-null  int64  
 1   start_time               295854 non-null  object 
 2   end_time                 295854 non-null  object 
 3   start_station_id         146286 non-null  float64
 4   start_station_name       146866 non-null  object 
 5   start_station_latitude   295854 non-null  float64
 6   start_station_longitude  295854 non-null  float64
 7   end_station_id           145934 non-null  float64
 8   end_station_name         146511 non-null  object 
 9   end_station_latitude     295854 non-null  float64
 10  end_station_longitude    295854 non-null  float64
 11  bike_id                  295854 non-null  int64  
 12  user_type                295854 non-null  object 
 13  rental_access_method     185746 non-null  object 
dtypes: f

In [7]:
df.duplicated().sum()

0

In [8]:
df.isnull().sum()

duration_sec                    0
start_time                      0
end_time                        0
start_station_id           149568
start_station_name         148988
start_station_latitude          0
start_station_longitude         0
end_station_id             149920
end_station_name           149343
end_station_latitude            0
end_station_longitude           0
bike_id                         0
user_type                       0
rental_access_method       110108
dtype: int64

In [9]:
df.nunique()

duration_sec                 5574
start_time                 282787
end_time                   282613
start_station_id              429
start_station_name            430
start_station_latitude     122218
start_station_longitude    123962
end_station_id                428
end_station_name              429
end_station_latitude       122941
end_station_longitude      125091
bike_id                      8016
user_type                       2
rental_access_method            2
dtype: int64

#### 2.1.2 categorical values

In [10]:
df['user_type'].value_counts()

Subscriber    170988
Customer      124866
Name: user_type, dtype: int64

In [11]:
df['rental_access_method'].value_counts()

app        171751
clipper     13995
Name: rental_access_method, dtype: int64

In [12]:
df['end_station_id'].value_counts()

30.0     5023
21.0     2888
81.0     2851
15.0     2835
58.0     2723
         ... 
472.0       5
234.0       5
228.0       5
421.0       3
435.0       1
Name: end_station_id, Length: 428, dtype: int64

In [13]:
df['start_station_id'].value_counts()

58.0     2753
30.0     2734
81.0     2638
22.0     2076
15.0     1998
         ... 
421.0       4
472.0       4
506.0       4
488.0       1
329.0       1
Name: start_station_id, Length: 429, dtype: int64

#### 2.1.2 ranges (numeric & dates)

In [14]:
df.describe()

,duration_sec,start_station_id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bike_id
count,295854.000000,146286.000000,295854.000000,295854.000000,145934.000000,295854.000000,295854.000000,295854.00000
mean,780.473193,168.900879,37.751277,-122.357866,161.061788,37.751261,-122.356994,321533.84539
std,2037.786317,141.956388,0.228337,0.650796,140.766154,0.238642,0.688512,307015.31534
min,60.000000,3.000000,0.000000,-122.513871,3.000000,0.000000,-122.513847,12.00000
25%,369.000000,53.000000,37.766214,-122.419481,43.000000,37.766376,-122.418306,11328.00000
50%,588.000000,120.000000,37.777158,-122.403234,113.000000,37.777370,-122.402717,279974.00000
75%,912.000000,263.000000,37.790102,-122.392450,250.000000,37.790492,-122.392546,545607.00000
max,811077.000000,506.000000,37.880222,0.000000,506.000000,37.927622,0.000000,999952.00000


In [15]:
df.start_time.min(), df.start_time.max()

('2020-01-01 00:05:19', '2020-01-31 23:59:48')

In [16]:
df.end_time.min(), df.end_time.max()

('2020-01-01 00:09:54', '2020-02-01 14:29:06.2630')

### 2.2 notes on intial findings re. the data:
***
Points marked `R` will first be researched on a deeper level, points marker `C` are ready for cleaning in the next step.

**Base trip data**
* (R) - Maximum duration in seconds is high (811k seconds = 225 hours) - something to investigate further. 
* (C) - Start and End time need to be converted to timestamps
* Note that our dataset spans the entire month of Jan-20
* (C) - user_type should be converted to a categorical type
* (C) - rental_access_method should be converted to categorical and change NULL to 'undefined'

**Location data - discreet**
* (C) - station_id's should be converted to integers
* (R) - there is a discrepancy between unique station_id's (429) vs name (430) - we should investigate this further.          

**Location data - continous**
* (R) - format is correct for Lat's and Long's, however we don't expect '0' to be included - we should investigate this further.
  



#### 2.2.1 further research into `duration_sec`
We should remove the duration of 811k seconds, this seems to be an error and only occurs once. Higest value below this is ~86k seconds (=1 day). Still high, but something to asses during EDA. 

In [17]:
# look at the case where duration is 811k seconds
df['duration_sec'].sort_values(ascending=False)

121168    811077
49926      86221
63677      85994
27024      85450
50984      85268
           ...  
202975        60
294590        60
196355        60
168851        60
232484        60
Name: duration_sec, Length: 295854, dtype: int64

In [18]:
df.loc[121168]

duration_sec                            811077
start_time                 2020-01-09 16:23:21
end_time                   2020-01-19 01:41:19
start_station_id                           NaN
start_station_name                         NaN
start_station_latitude                  37.336
start_station_longitude                 -121.9
end_station_id                             NaN
end_station_name                           NaN
end_station_latitude                   37.3365
end_station_longitude                 -121.901
bike_id                                 231296
user_type                           Subscriber
rental_access_method                   clipper
Name: 121168, dtype: object

#### 2.2.2 further research into `station_id` and `station_name`
It seems the ID for `Steuart St at Market St` is missing, affecting 1155 rows. We will not adjust this, we do make a note that for any grouping by station we best use name or ignore this station.

In [19]:
# descrepancy between ID and name for stations
df.groupby('start_station_name')['start_station_id'].max().reset_index().sort_values('start_station_id').tail()


,start_station_name,start_station_id
191,Frida Kahlo Way at Cloud Cir,498.0
306,Otis St at Brady St,499.0
103,Battery St at Filbert St,500.0
77,7th Ave at Clement St,506.0
376,Steuart St at Market St,NaN


In [20]:
df.groupby('end_station_name')['end_station_id'].max().reset_index().sort_values('end_station_id').tail()


,end_station_name,end_station_id
191,Frida Kahlo Way at Cloud Cir,498.0
306,Otis St at Brady St,499.0
103,Battery St at Filbert St,500.0
77,7th Ave at Clement St,506.0
375,Steuart St at Market St,NaN


In [21]:
df[(df['start_station_name'] == 'Steuart St at Market St') | (df['end_station_name'] == 'Steuart St at Market St')].shape

(1155, 14)

#### 2.2.3 further research into `latitude` and `longitude`
Only a few rows have '0,0' as value here (8 for start_station, 9 for end_station) and these should be removed. Other location seems to be in reasonable values.

In [22]:
## lat and long
df.groupby('start_station_longitude').size().reset_index().sort_values('start_station_longitude', ascending=False).head(2)

,start_station_longitude,0
123961,0.000000,8
123960,-121.809475,1


In [23]:
df.groupby('start_station_latitude').size().reset_index().sort_values('start_station_latitude', ascending=True).head(2)

,start_station_latitude,0
0,0.000000,8
1,37.275291,1


In [24]:
df.groupby('end_station_longitude').size().reset_index().sort_values('end_station_longitude', ascending=False).head(2)

,end_station_longitude,0
125090,0.000000,9
125089,-121.772722,1


In [25]:
df.groupby('end_station_latitude').size().reset_index().sort_values('end_station_latitude', ascending=True).head(2)

,end_station_latitude,0
0,0.000000,9
1,37.258978,1


### 2.3 Final Assessment notes
***
Issues to address:
1. Remove record with `duration_sec` 811077 (index=121168)
2. Convert `start_time` and `end_time` to datetime format.
3. Convert `start_station_id` to integer
4. Remove records where `start_station_latitude` and/or `start_station_longitude` is null (8 in total)
5. Convert `end_station_id` to integer 
6. Remove records where `end_station_latitude` and/or `end_station_longitude` is null (9 in total)
7. Convert `user_type` to a categorical
8. Convert `rental_access_method` to a categorical, where `nan` is changed to `undefined`

Notes:
* To group by station, we should use the `name` fields, as the id for `Steuart St at Market St` is missing.
* Our dataset spans the entire month of Jan-2020
* Locations seems to be in the US (expected since data is from LYFT)

## 3. Clean Data
***
Addressing the issues mentioned above in order. Each time, following the plan-code-test structure.

In [26]:
# copy dataset before starting
df_clean = df.copy()
print(df_clean.shape)

(295854, 14)


#### 3.1 Remove record with `duration_sec` 811077 (index=121168)
Longer than 24h rent results in a fine regardless.

In [27]:
# only keep rows where duration_sec < 100,000
df_clean = df_clean[df_clean['duration_sec'] < 100000].reset_index(drop=True)

In [28]:
assert len(df_clean[df_clean['duration_sec'] > 100000])==0
print(df_clean.shape)

(295853, 14)


#### 3.2 Convert `start_time` and `end_time` to datetime format.

In [29]:
# start_time (check reveals: dtype: datetime64[ns])
df_clean['start_time'] = pd.to_datetime(df_clean['start_time'])
df_clean['start_time']

0        2020-01-31 15:23:47.733
1        2020-01-31 15:40:31.616
2        2020-01-31 17:48:16.635
3        2020-01-31 17:53:03.413
4        2020-01-31 17:12:33.460
                   ...          
295848   2020-01-26 16:18:12.000
295849   2020-01-06 12:40:12.000
295850   2020-01-09 13:56:54.000
295851   2020-01-30 23:08:59.000
295852   2020-01-04 15:11:49.000
Name: start_time, Length: 295853, dtype: datetime64[ns]

In [30]:
# start_time (check reveals: dtype: datetime64[ns])
df_clean['end_time'] = pd.to_datetime(df_clean['end_time'])
df_clean['end_time']

0        2020-02-01 14:29:06.263
1        2020-02-01 10:37:51.000
2        2020-02-01 09:11:44.317
3        2020-02-01 09:08:32.634
4        2020-02-01 08:40:53.646
                   ...          
295848   2020-01-26 16:33:02.000
295849   2020-01-06 13:02:20.000
295850   2020-01-09 17:25:44.000
295851   2020-01-30 23:37:54.000
295852   2020-01-04 16:59:47.000
Name: end_time, Length: 295853, dtype: datetime64[ns]

#### 3.3 Convert `start_station_id` to integer


In [31]:
# set 'nan' to '-1' to allow integer conversion
df_clean['start_station_id'] = df_clean['start_station_id'].fillna('-1')
df_clean['start_station_id'] = df_clean['start_station_id'].astype('int')

In [32]:
# visual check
df_clean['start_station_id']

0         400
1          99
2         197
3         197
4          12
         ... 
295848     -1
295849     -1
295850     -1
295851     -1
295852     -1
Name: start_station_id, Length: 295853, dtype: int64

#### 3.4 Remove records where `start_station_latitude` and/or `start_station_longitude` is null (8 in total)


In [33]:
# before
df_clean.shape

(295853, 14)

In [34]:
# only select rows where lat/lon is not 0
df_clean = df_clean[(df_clean['start_station_latitude'] > 0)  | (df_clean['start_station_longitude'] < 0)]

In [35]:
# test
assert len(df_clean[(df_clean['start_station_latitude'] == 0)  | (df_clean['start_station_longitude'] == 0)])==0
print(df_clean.shape)

(295845, 14)


#### 3.5 Convert `end_station_id` to integer 


In [36]:
# set 'nan' to '-1' to allow integer conversion
df_clean['end_station_id'] = df_clean['end_station_id'].fillna('-1')
df_clean['end_station_id'] = df_clean['end_station_id'].astype('int')

In [37]:
# visual check
df_clean['end_station_id']

0         400
1          99
2         197
3         197
4         371
         ... 
295848     -1
295849     -1
295850    468
295851     -1
295852     -1
Name: end_station_id, Length: 295845, dtype: int64

#### 3.6 Remove records where `end_station_latitude` and/or `end_station_longitude` is null (9 in total)


In [38]:
#before
df_clean.shape

(295845, 14)

In [39]:
# only select rows where lat/lon is not 0
df_clean = df_clean[(df_clean['end_station_latitude'] > 0)  | (df_clean['end_station_longitude'] < 0)]

In [40]:
# test
assert len(df_clean[(df_clean['end_station_latitude'] == 0)  | (df_clean['end_station_longitude'] == 0)])==0
print(df_clean.shape)

(295836, 14)


#### 3.7 Convert `user_type` to a categorical


In [41]:
df_clean['user_type'].value_counts()

Subscriber    170978
Customer      124858
Name: user_type, dtype: int64

In [42]:
# convert
df_clean['user_type'] = df_clean['user_type'].astype('category')

# visual check
df_clean['user_type']

0           Customer
1         Subscriber
2           Customer
3           Customer
4           Customer
             ...    
295848      Customer
295849    Subscriber
295850    Subscriber
295851    Subscriber
295852      Customer
Name: user_type, Length: 295836, dtype: category
Categories (2, object): [Customer, Subscriber]

#### 3.8 Convert `rental_access_method` to a categorical, where `nan` is changed to `undefined`


In [43]:
# get categories
df_clean['rental_access_method'].value_counts()

app        171743
clipper     13993
Name: rental_access_method, dtype: int64

In [44]:
# undefined category
df_clean['rental_access_method'].isnull().sum()

110100

In [45]:
# add undefined category
df_clean['rental_access_method'].replace(np.nan, 'undefined', inplace=True)

In [46]:
# convert
df_clean['rental_access_method'] = df_clean['rental_access_method'].astype('category')

# visual check
df_clean['rental_access_method']

0         undefined
1         undefined
2         undefined
3         undefined
4         undefined
            ...    
295848          app
295849          app
295850          app
295851          app
295852          app
Name: rental_access_method, Length: 295836, dtype: category
Categories (3, object): [app, clipper, undefined]

In [47]:
# get categories - check
df_clean['rental_access_method'].value_counts()

app          171743
undefined    110100
clipper       13993
Name: rental_access_method, dtype: int64

#### 3.9 Overal check data-types
***

In [48]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295836 entries, 0 to 295852
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   duration_sec             295836 non-null  int64         
 1   start_time               295836 non-null  datetime64[ns]
 2   end_time                 295836 non-null  datetime64[ns]
 3   start_station_id         295836 non-null  int64         
 4   start_station_name       146850 non-null  object        
 5   start_station_latitude   295836 non-null  float64       
 6   start_station_longitude  295836 non-null  float64       
 7   end_station_id           295836 non-null  int64         
 8   end_station_name         146502 non-null  object        
 9   end_station_latitude     295836 non-null  float64       
 10  end_station_longitude    295836 non-null  float64       
 11  bike_id                  295836 non-null  int64         
 12  user_type       

## 4. store cleaned data
***
We store the cleaned data in as hdf file to keep the formatting (categories, datetime) intact.

#### 4.1 store data

In [51]:
# store as hdf
path = 'data/202001-tripdata-cleaned.h5'
df_clean.to_hdf(path, key='bay_wheels', format='table', mode='w')


In [52]:
df_check = pd.read_hdf(path)
df_check.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295836 entries, 0 to 295852
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   duration_sec             295836 non-null  int64         
 1   start_time               295836 non-null  datetime64[ns]
 2   end_time                 295836 non-null  datetime64[ns]
 3   start_station_id         295836 non-null  int64         
 4   start_station_name       146850 non-null  object        
 5   start_station_latitude   295836 non-null  float64       
 6   start_station_longitude  295836 non-null  float64       
 7   end_station_id           295836 non-null  int64         
 8   end_station_name         146502 non-null  object        
 9   end_station_latitude     295836 non-null  float64       
 10  end_station_longitude    295836 non-null  float64       
 11  bike_id                  295836 non-null  int64         
 12  user_type       

#### 4.2 outlook
The process outlined in this notebook can easily be repeated for data from other months. The first few times, we should carefully observe if we can see any issues that we didn't see in this file, but once we are confident we have a stable method of procedssing, we can create a single function to do the processing to act on the data of each new month. 